### Topic Modeling using HDP and LDA

- Text Processing
- Generating dictionary of vocabulary
- Mapping corpus using dictionary
- Training the Topic Model

In [1]:
!pip install pyLDAvis


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy
#시간이 좀 걸려요

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Rea

In [3]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"


env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [4]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996
#시간이 좀 걸립니다.

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /tmp/mecab-python-0.996
  Created wheel for mecab-python: filename=mecab_python-0.996_ko_0.9.2-cp37-cp37m-linux_x86_64.whl size=141808 sha256=a1b3f1be7966674760a7ef344d732e44c846c0d55d637c6a3feb6d648afe0bfc
  Stored in directory: /root/.cache/pip/wheels/40/7b/9f/2922869bef86c3354ae7034f7a3647c573ee1997c2dad0290a
Failed to build mecab-python
  Attempting uninstall: mecab-python
    Found existing installation: mecab-python 0.996-ko-0.9.2
    Uninstalling mecab-python-0.996-ko-0.9.2:
      Successfully uninstalled mecab-python-0.996-ko-0.9.2
    Running setup.py install for mecab-python: started
    Running setup.py install for mecab-python: finished with status 'done'


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  DEPRECATION: mecab-python was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from konlpy.tag import Mecab

In [7]:
import matplotlib.pyplot as plt
import gensim
import numpy as np
import spacy
import pandas as pd
import re

from gensim import corpora, models
from gensim.models.ldamodel import LdaModel
from spacy.lang.en.stop_words import STOP_WORDS
import pyLDAvis.gensim_models
#Import nltk stopwords and add custom stopwords that are likely to appear in news articles.
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(["mrs","ms","say","he","mr","she","they","company"])

import os, re, operator, warnings
warnings.filterwarnings('ignore')
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [8]:
df=pd.read_csv("NewsArticles.csv", encoding='unicode_escape',index_col=0)
#drop all the unnamed columns
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df.head()

,publish_date,article_source_link,title,subtitle,text
article_id,,,,,
1,2017/2/7,http://abcnews.go.com/Politics/pence-break-tie...,"Betsy DeVos Confirmed as Education Secretary, ...",NaN,Michigan billionaire education activist Betsy ...
2,2017/2/7,http://abcnews.go.com/Politics/wireStory/melan...,Melania Trump Says White House Could Mean Mill...,NaN,First lady Melania Trump has said little about...
3,2017/2/7,http://abcnews.go.com/Politics/wireStory/trump...,"As Trump Fears Fraud, GOP Eliminates Election ...",NaN,A House committee voted on Tuesday to eliminat...
4,2017/2/7,http://abcnews.go.com/Politics/appeals-court-d...,Appeals Court to Decide on Challenge to Trump'...,NaN,"This afternoon, three federal judges from the ..."
5,2017/2/7,http://abcnews.go.com/US/23-states-winter-weat...,At Least 4 Tornadoes Reported in Southeast Lou...,NaN,At least four tornadoes touched down in Louisi...


In [9]:
# before loading the language you have to download it first. Go to your command prompt and execute this statement and 
# restart the kernel:
# python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

In [10]:
data = df['text'].values.tolist()


In [11]:
#removing punctuations and others characters
def preprocess(string):
    return re.sub('[^\w_\s-]', ' ',str(string))

data = list(map(preprocess,data))  

In [12]:
#data cleaning and lemmatization
lemma_doc = []
for datum in data:
    sent = nlp(str(datum).lower())
    text = []
    for w in sent:
        if not w.is_stop and not w.is_punct and not w.like_num and str(w) not in stop_words and (len(str(w)) > 4):
            #adding the lematized version of the words
            text.append(w.lemma_)
    lemma_doc.append(text)
    

In [13]:
lemma_doc[0]


['michigan',
 'billionaire',
 'education',
 'activist',
 'betsy',
 'devos',
 'confirm',
 'today',
 'serve',
 'secretary',
 'education',
 'president',
 'trump',
 'administration',
 'president',
 'pence',
 'break',
 'senate',
 'senate',
 'vote',
 'devos',
 'highly',
 'contentious',
 'nomination',
 'afternoon',
 'tally',
 'split',
 'evenly',
 'require',
 'penny',
 'authority',
 'president',
 'upper',
 'chamber',
 'congress',
 'break',
 'impasse',
 'president',
 'break',
 'confirm',
 'cabinet',
 'nominee',
 'penny',
 'count',
 'vote',
 'render',
 'tally',
 'democrats',
 'stage',
 'marathon',
 'speech',
 'lawmaker',
 'take',
 'floor',
 'additional',
 'republican',
 'devos',
 'block',
 'confirmation',
 'imagine',
 'bad',
 'choice',
 'elizabeth',
 'warren',
 'letter',
 'constituent',
 'urge',
 'devos',
 'stir',
 'vehement',
 'opposition',
 'teacher',
 'union',
 'senate',
 'democrats',
 'cite',
 'concern',
 'support',
 'school',
 'voucher',
 'critic',
 'believe',
 'weaken',
 'public',
 'school

In [14]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(lemma_doc, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[lemma_doc], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[lemma_doc[0]]])

['michigan', 'billionaire', 'education', 'activist', 'betsy_devos', 'confirm', 'today', 'serve', 'secretary', 'education', 'president', 'trump', 'administration', 'president', 'pence', 'break', 'senate', 'senate', 'vote', 'devos', 'highly', 'contentious', 'nomination', 'afternoon', 'tally', 'split', 'evenly', 'require', 'penny', 'authority', 'president', 'upper', 'chamber', 'congress', 'break', 'impasse', 'president', 'break', 'confirm', 'cabinet_nominee', 'penny', 'count', 'vote', 'render', 'tally', 'democrats', 'stage', 'marathon', 'speech', 'lawmaker', 'take', 'floor', 'additional', 'republican', 'devos', 'block', 'confirmation', 'imagine', 'bad', 'choice', 'elizabeth_warren', 'letter', 'constituent', 'urge', 'devos', 'stir', 'vehement', 'opposition', 'teacher', 'union', 'senate', 'democrats', 'cite', 'concern', 'support', 'school', 'voucher', 'critic', 'believe', 'weaken', 'public', 'school', 'experience', 'attend', 'work', 'public', 'education', 'system', 'cite', 'familiarity', 'l

**Create the Dictionary and Corpus needed for Topic Modeling**

- Word to IDs mapping
- Bag of words of each document
- corpus (cluster of Bag of words of all the documents)

In [15]:
#Creates Word to IDs mapping
word2id = corpora.Dictionary(lemma_doc)

In [16]:
# Creates bag of words and a corpus
documents = lemma_doc
corpus = [word2id.doc2bow(doc) for doc in documents]

print('Corpus sample')
sample = corpus[0]
for i in range(len(sample)):
    print('Word', sample[i][0], ':', word2id[sample[i][0]], ' || Number of occurences:', sample[i][1])

Corpus sample
Word 0 :        || Number of occurences: 1
Word 1 : account  || Number of occurences: 1
Word 2 : activist  || Number of occurences: 1
Word 3 : addition  || Number of occurences: 1
Word 4 : additional  || Number of occurences: 1
Word 5 : administration  || Number of occurences: 1
Word 6 : administrator  || Number of occurences: 1
Word 7 : afternoon  || Number of occurences: 1
Word 8 : alaska  || Number of occurences: 1
Word 9 : announce  || Number of occurences: 2
Word 10 : answer  || Number of occurences: 1
Word 11 : appreciate  || Number of occurences: 1
Word 12 : attend  || Number of occurences: 1
Word 13 : authority  || Number of occurences: 1
Word 14 : average  || Number of occurences: 1
Word 15 : bad  || Number of occurences: 1
Word 16 : believe  || Number of occurences: 1
Word 17 : betsy  || Number of occurences: 1
Word 18 : billionaire  || Number of occurences: 1
Word 19 : block  || Number of occurences: 1
Word 20 : board  || Number of occurences: 1
Word 21 : break

Hierarchical Dirichlet Processing

This is kind of an unsupervised technique (Topic modeling is a unsupervised technique. Here the context is we don't decide the # of topics. 

In concept this is similar to Hierarchical cluster as don't choose the number of cluster before hand) as the model will identify the number of topics. Let's see what it will produce.

In [17]:
hdp = models.HdpModel(corpus,word2id)
hdp_topics = hdp.print_topics()
for topic in hdp_topics:
    print(topic)

(0, '0.008*trump + 0.006*people + 0.005*president + 0.004*country + 0.004*government + 0.004*state + 0.004*china + 0.003*house + 0.003*report + 0.003*include')
(1, '0.007*trump + 0.006*people + 0.005*country + 0.005*president + 0.004*china + 0.004*government + 0.003*state + 0.003*report + 0.003*house + 0.003*year')
(2, '0.010*china + 0.004*chinese + 0.003*percent + 0.003*market + 0.003*country + 0.003*world + 0.002*government + 0.002*chestnut + 0.002*european + 0.002*growth')
(3, '0.006*china + 0.004*happen + 0.004*cover + 0.004*people + 0.003*attack + 0.003*trump + 0.003*president + 0.002*country + 0.002*police + 0.002*house')
(4, '0.003*china + 0.002*people + 0.002*child + 0.002*waste + 0.002*court + 0.002*year + 0.002*country + 0.002*family + 0.002*service + 0.002*government')
(5, '0.004*china + 0.002*chinese + 0.002*trump + 0.002*court + 0.002*government + 0.002*country + 0.002*people + 0.002*state + 0.001*domestic + 0.001*percent')
(6, '0.003*china + 0.002*country + 0.002*people +

In [18]:
print('HDP model created: '+str(len(hdp_topics))+' Topics')


HDP model created: 20 Topics


## Latent Dirichlet Allocation Model


In [19]:
lda_model = LdaModel(corpus=corpus, id2word=word2id, num_topics=5, random_state=42, update_every=1, chunksize=100, 
                     passes=10, alpha='auto', per_word_topics=True)


In [20]:
#Article - Topic Distribution for first Article
def get_article_topic_distribution(article):
    return lda.get_document_topics(article)
#Returns a list containing a list of tuple
#Each inner list corresponds to an article and each tuple refers to topicID and its corresponding probability  
map(get_article_topic_distribution, corpus)

In [21]:
lda_model.print_topics()


[(0,
  '0.020*"trump" + 0.012*"president" + 0.011*"house" + 0.007*"committee" + 0.006*"state" + 0.006*"government" + 0.006*"white" + 0.005*"party" + 0.005*"election" + 0.005*"campaign"'),
 (1,
  '0.012*"russian" + 0.012*"russia" + 0.011*"country" + 0.009*"north" + 0.009*"government" + 0.008*"force" + 0.007*"korea" + 0.006*"military" + 0.006*"state" + 0.006*"group"'),
 (2,
  '0.010*"china" + 0.009*"percent" + 0.006*"country" + 0.006*"business" + 0.006*"market" + 0.005*"trade" + 0.004*"company" + 0.004*"economic" + 0.004*"european" + 0.004*"project"'),
 (3,
  '0.015*"police" + 0.010*"garda" + 0.007*"people" + 0.007*"court" + 0.006*"arrest" + 0.006*"authority" + 0.006*"officer" + 0.006*"report" + 0.006*"death" + 0.005*"service"'),
 (4,
  '0.010*"woman" + 0.010*"people" + 0.007*"family" + 0.006*"child" + 0.005*"school" + 0.005*"young" + 0.004*"year" + 0.004*"friend" + 0.004*"think" + 0.004*"learn"')]

**How to interpret this?**

The top 10 keywords that contribute to the topic are showcased with their respective weight.

Let's try to interpret the 5 topics:

Topic 1: key words like "Russia", "Country", "Government", "Minister" suggest Politics in Russia

Topic 2: key words like "China", "Brexit","Trade", "Business", 
"Market" suggest Inter country trade news

Topic 3: key words like "Player","Sport","World" suggest Sports news (football)

Topic 4: key words like "People","Woman","Police", "Family,"Child" suggest Domestic news

Topic 5: key words like "Trump", "State", "White", "Committee" suggest Polictics in USA

### Compute Model Perplexity and Coherence Score


In [22]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = models.CoherenceModel(model=lda_model, texts=lemma_doc, dictionary=word2id, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.408782266100435

Coherence Score:  0.47351504028371866


Coherence measures the relative distance between words within a topic.

There are two major types C_V typically 0 < x < 1 and uMass -14 < x < 14. 

Coherence score of 0.4 is low. I want to explore what would have been the ideal number of topics. Will explore the elbow method below.

In [23]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model=LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42, update_every=1, chunksize=100, 
                     passes=10, alpha='auto', per_word_topics=True)
        model_list.append(model)
        coherencemodel = models.CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
#Use this to get the graph of optimal # of topics
model_list, coherence_values = compute_coherence_values(dictionary=word2id, corpus=corpus, texts=lemma_doc, start=2, limit=100, step=10)
# Show graph
import matplotlib.pyplot as plt
limit=100; start=2; step=10;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

You can decide on the number of topics based on this analysis. 

Note that the Customization used for 5 topic model (lda_model) and the optimization models is difference therefore the Coherence score for 5 topics LDA model differ.

### Vizualize the topics


In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model,corpus,word2id)